In [ ]:
import pandas as pd
import os
from io import StringIO, BytesIO
from datetime import datetime
from pandas import Series
from lxml import etree

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200

In [ ]:
import xml.etree.ElementTree as ET

class XML2DataFrame:

    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)
    
def select_ints(item):
    try:
        int(item)
        return True
    except:
        return False
    
def f(x):
     return Series(dict(qty = x['qty'].sum(), 
                        siz = "{%s}" % ', '.join(x['siz'])))

In [ ]:
for i in range(7):
    print('Loading data for Inventory00{}, {}'.format(i, datetime.now()))
    tree = etree.parse("Inventory00{}.xml".format(i))
    
    print('Converting Inventory00{} to a df, {}'.format(i, datetime.now()))
    df = pd.DataFrame()
    for item in tree.findall('.//INVENTORY'):
        table = XML2DataFrame(etree.tostring(item)).process_data()
        if table.loc[2, 'dcs_code'] == 'W  S':
            df = df.append(table.loc[2])
    df = df.dropna(axis=0, how='all')
    
    print('Selecting relevant data from Inventory00{}, {}'.format(i, datetime.now()))
    items = df[['dcs_code', 'vend_code', 'description1', 'attr', 'siz', 'active', 'qty', 'sold_qty', 'cost', 'price', 'lst_sold_date', 'lst_rcvd_date', 'fst_rcvd_date']]
    items.sort_values(['vend_code', 'description1', 'attr', 'siz'], ascending=True)
    
    sandals = items[items['dcs_code'] == 'W  S']
    sum_info = sandals[['vend_code', 'description1', 'attr', 'qty', 'siz']]
    sum_info = sum_info.loc[[select_ints(item) for item in sum_info['qty']]]
    sum_info['qty'] = sum_info['qty'].astype(int)
    sum_info = sum_info.loc[sum_info['qty'] > 0]
    sum_info['siz'] = sum_info['siz'].astype(str)
    print(sum_info.head())
    sums = sum_info.groupby(['vend_code', 'description1', 'attr']).apply(f)
    sumz = sums.reset_index()
    print(sumz.head())
    
    print('Calculating small sums from Inventory00{}, {}'.format(i, datetime.now()))
    small_sums = sumz.loc[[val <=2 for val in sumz['qty']]]
    small_sums.to_csv('/Users/nathansuberi/Desktop/enchanted_reports/inventory_analysis/small_sums_from_inventory00{}.csv'.format(i))

In [ ]:
loc = '/Users/nathansuberi/Desktop/enchanted_reports/inventory_analysis/small_sums_from_inventory00{}.csv'
df = pd.DataFrame()
for i in range(7):
    _df = pd.read_csv(loc.format(i))
    df = df.append(_df)

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
f

In [ ]:
df = df.sort_values(['vend_code', 'description1', 'attr'])

In [ ]:
final_sums = df.groupby(['vend_code', 'description1', 'attr']).apply(f).reset_index()

In [ ]:
final_sums[final_sums['qty'] <= 2].to_csv('/Users/nathansuberi/Desktop/enchanted_reports/inventory_analysis/final_sums.csv')
final_sums[final_sums['qty'] <= 2].to_html('/Users/nathansuberi/Desktop/enchanted_reports/inventory_analysis/final_sums.html')

In [ ]:
vendors = df.groupby(['vend_code', 'description1', 'attr']).apply(f).reset_index()['vend_code'].unique()

In [ ]:
vendors

In [ ]:
exclude_list = ['ABO', 'ACO', 'WOL', 'XSE', 'ZEE',
                'AGS', 'AHN', 'ANA', 'ANT', 'ARA', 'ARI',
                'BAN', 'BLE', 'BLU', 'BRI', 'CCR', 'CHI',
                'CHO', 'CLA', 'COR', 'DEC', 'DOL', 'DOU',
                'DRE', 'ELN', 'FID', 'FLE', 'FLX', 
                'FRY', 'GEO', 'GOL', 'HAV', 'HEL', 'HIS',
                'JOB', 'JUI', 'LIZ', 'MAC', 'MBT', 'MAD',
                'MOD', 'MOI', 'MUN', 'MUR', 'OKA', 'OTB',
                'PEN', 'PEP', 'POR', 'PRE', 'QWA', 'RAF', 
                'RAI', 'RZD', 'SAN', 'SCF', 'SEN', 'SID',
                'SPE', 'STO', 'SWI', 'TAB', 'TAM', 'THE',
                'TIC', 'TOM', 'VAL', 'VER', 'VIC', 'VID',
                'VIO', 'VOL']

In [ ]:
#final_sums = df.groupby(['vend_code', 'description1', 'attr']).apply(f)
with open('/Users/nathansuberi/Desktop/enchanted_reports/inventory_analysis/final_sums_reference.html', 'w') as f:
    for vendor in vendors:
        vendor_data = final_sums.loc[vendor, :, :].reset_index()
        vendor_data = vendor_data[vendor_data['qty'] <= 2]

        if vendor_data.shape[0] and vendor not in exclude_list:
            f.write('<h1>Vendor: ' + vendor + '</h1>\n')
            last_style = ''
            for line in vendor_data.values:
                style = line[1]
                color = line[2]
                sizes = line[4].replace('{', '').replace('}', '')
                if last_style == style:
                    f.write('</br><i>{}, {}, sizes: <font size=4>{}</font></i>'.format(style, color, sizes) + '</i>')
                else:
                    f.write('<br/><b>{}, {}, sizes: <font size=4>{}</font></b>'.format(style, color, sizes) + '</b>')
                last_style = style

In [ ]:
z = zip(items['description1'].str.contains('HELEN'), items['attr'].str.contains('27577'))
flag = [a and b for a,b in z]
items[flag]

Notes:
http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

In [ ]:
with open("Inventory006.xml", "r") as f:
    xml_data = f.read()
#xml2df = XML2DataFrame(xml_data)
xml2df = XML2DataFrame(etree.tostring())
xml_dataframe = xml2df.process_data()